In [1]:
spark

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 59, 6, Finished, Available, Finished)

In [2]:
from azureml.core import Workspace, Dataset, Datastore

subscription_id = '21ff0fc0-dd2c-450d-93b7-96eeb3699b22'
resource_group = 'project-group-29'
workspace_name = 'project-group-29'

workspace = Workspace(subscription_id, resource_group, workspace_name)

datastore = Datastore.get(workspace, "workspaceblobstore")
dataset = Dataset.Tabular.from_parquet_files(path=(datastore, 'states/states_sentiment'))
df = dataset.to_spark_dataframe()
df.head()

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 59, 7, Finished, Available, Finished)

{'infer_column_types': 'False', 'activity': 'to_spark_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_spark_dataframe', 'activityApp': 'TabularDataset'}
{'infer_column_types': 'False', 'activity': 'to_spark_dataframe', 'activityApp': 'TabularDataset', 'runId': '2b7c5281-0c7f-49e8-9d40-e9b4483b4fd4'}
{'infer_column_types': 'False', 'activity': 'to_spark_dataframe', 'activityApp': 'TabularDataset', 'runId': '2b7c5281-0c7f-49e8-9d40-e9b4483b4fd4', 'run_id': '2b7c5281-0c7f-49e8-9d40-e9b4483b4fd4'}


Row(author='fireweedflowers', text='I live in Colorado these days but I was born and raised in Wyoming and was emphatically a socialist.', controversiality=0, created_utc=1701619261, distinguished=None, edited=None, gilded=0, id='kbtqz1h', parent_id='t3_187jmr5', score=1, subreddit='wyoming', subreddit_id='t5_2r53d', alaska=False, alabama=False, arkansas=False, american samoa=False, arizona=False, california=False, colorado=True, connecticut=False, district of columbia=False, delaware=False, florida=False, georgia=False, guam=False, hawaii=False, iowa=False, idaho=False, illinois=False, indiana=False, kansas=False, kentucky=False, louisiana=False, massachusetts=False, maryland=False, maine=False, michigan=False, minnesota=False, missouri=False, mississippi=False, montana=False, north carolina=False, north dakota=False, nebraska=False, new hampshire=False, new jersey=False, new mexico=False, nevada=False, new york=False, ohio=False, oklahoma=False, oregon=False, pennsylvania=False, puer

In [3]:
#df.printSchema()
df.show()

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 55, 8, Finished, Available, Finished)

+--------------------+--------------------+----------------+-----------+-------------+------+------+-------+----------+-----+-------------------+------------+------+-------+--------+--------------+-------+----------+--------+-----------+--------------------+--------+-------+-------+-----+------+-----+-----+--------+-------+------+--------+---------+-------------+--------+-----+--------+---------+--------+-----------+-------+--------------+------------+--------+-------------+----------+----------+------+--------+-----+--------+------+------------+-----------+------------+--------------+------------+---------+-----+-----+--------+--------------+-------+----------+---------+-------------+-------+-------------+
|              author|                text|controversiality|created_utc|distinguished|edited|gilded|     id| parent_id|score|          subreddit|subreddit_id|alaska|alabama|arkansas|american samoa|arizona|california|colorado|connecticut|district of columbia|delaware|florida|georgia|

In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, VectorAssembler
from pyspark.ml.regression import LinearRegression

state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", 
"Colorado", "Connecticut", "District of Columbia", "Delaware", "Florida", "Georgia", "Guam", 
"Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", 
"Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", 
"Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", 
"New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", 
"Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", 
"Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]
state_names = [x.lower() for x in state_names]

state_names.append("sentiment_num")
state_names.append("rawFeatures")

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=1000)
idf = VectorAssembler(inputCols= state_names, outputCol="features")

lr = LinearRegression(featuresCol="features", labelCol="score")

pipeline_linear = Pipeline(stages=[tokenizer, hashingTF, idf, lr])

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 55, 13, Finished, Available, Finished)

In [10]:
(trainingData, testData) = df.randomSplit([0.7, 0.3])

model_linear = pipeline_linear.fit(trainingData)
predictions_linear = model_linear.transform(testData)

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 55, 15, Finished, Available, Finished)

In [18]:
datastore = 'azureml://datastores/workspaceblobstore/paths'
model_linear.write().overwrite().save(f"{datastore}/models/ml_model_linear_score_prediction")

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 55, 23, Finished, Available, Finished)

In [12]:
from pyspark.ml import PipelineModel
from pyspark.ml.evaluation import RegressionEvaluator

PipelineModel.load(f"{datastore}/models/ml_model_score_prediction") 
evaluator = RegressionEvaluator(labelCol="score", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions_linear)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 55, 17, Finished, Available, Finished)

Root Mean Squared Error (RMSE) on test data = 68.5895


In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import PipelineModel
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 59, 8, Finished, Available, Finished)

In [4]:
state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", 
"Colorado", "Connecticut", "District of Columbia", "Delaware", "Florida", "Georgia", "Guam", 
"Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", 
"Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", 
"Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", 
"New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", 
"Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", 
"Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]
state_names = [x.lower() for x in state_names]

state_names.append("sentiment_num")
state_names.append("rawFeatures")

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=1000)
idf = VectorAssembler(inputCols= state_names, outputCol="features")

rf = RandomForestRegressor(featuresCol="features", labelCol = "score")

pipeline_forest = Pipeline(stages=[tokenizer, hashingTF, idf, rf])

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 59, 9, Finished, Available, Finished)

In [5]:
(trainingData, testData) = df.randomSplit([0.7, 0.3])

model_forest = pipeline_forest.fit(trainingData)
predictions_forest = model_forest.transform(testData)

datastore = 'azureml://datastores/workspaceblobstore/paths'
model_forest.write().overwrite().save(f"{datastore}/models/ml_model_forest_score_prediction")

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 59, 10, Submitted, Running, Running)

In [6]:
#PipelineModel.load(f"{datastore}/models/ml_model_score_prediction") 
evaluator = RegressionEvaluator(labelCol="score", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions_forest)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

StatementMeta(, , , Waiting, , Waiting)

KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages/py4j/java_gateway.py", line 1217, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/trusted-service-user/cluster-env/env/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
job4_comments = spark(
    description="states_averages",
    code="./",
    entry={"file": "ml_forest_model.py"},
    driver_cores=1,
    driver_memory="7g",
    executor_cores=4,
    executor_memory="7g",
    executor_instances=6,
    resources={
        "instance_type": "Standard_E4S_V3",
        "runtime_version": "3.4",
    },
    inputs={
        "input_data": Input(
            type="uri_folder",
            path="azureml://datastores/workspaceblobstore/paths/states/states_sentiment",
            mode="direct",
        ),
    },
    environment="sparknlp-python-env@latest",
    identity=UserIdentityConfiguration(),
    args="--input_data ${{inputs.input_data}}"
)


In [ ]:
job_ml_object = azureml_client.jobs.create_or_update(job_ml_comments)